# TMVA Convolutional Neural Network tutorial

This is a tutorial explaining how to train a simple Convolutional Neural Network, given the ECAL images dataset.
The dataset is consisted of several branches, where each event is a flat array of 1024 elements. That means, each image is a 32 by 32 pixels with only one channel. One event is either classified as a photon or electron.

Therefore, we create a Convolutional Neural Network consisted of the following layers: Convolutional Layer with 12 2 by 2 filters, striding and zero padding one in both dimensions, Max Pool Layer with 6 by 6 frames and striding 1 in both dimensions, and finally two Dense Layers with 512 and 32 units respectively.

### Load the photon input

In [1]:
TMVA::Tools::Instance();

// Load the photon input
TFile *photonInput(0);
TString photonFileName = "dataset/SinglePhotonPt50_FEVTDEBUG_n250k_IMG_CROPS32.root";
photonInput = TFile::Open(photonFileName);

### Load the electron input

In [2]:
// Load the electron input
TFile *electronInput(0);
TString electronFileName = "dataset/SingleElectronPt50_FEVTDEBUG_n250k_IMG_CROPS32.root";
electronInput = TFile::Open(electronFileName);

### Create the output file and the factory object

In [3]:
// Create a ROOT output file where TMVA will store ntuples, histograms, etc.
TString outfileName("results/TMVA_MethodDL.root");
TFile *outputFile = TFile::Open(outfileName, "RECREATE");

// Create a factory for booking the method
TMVA::Factory *factory = new TMVA::Factory("TMVAClassification", outputFile,
                        "!V:!Silent:Color:DrawProgressBar:Transformations=I:AnalysisType=Classification");

### Create data loader object and assign the appropriate trees

In [4]:
TMVA::DataLoader *dataloader = new TMVA::DataLoader("dataset");

In [5]:
// Get the trees
TTree *signalTree = (TTree *)photonInput->Get("RHTree");
TTree *background = (TTree *)electronInput->Get("RHTree");

// global event weights per tree
Double_t signalWeight = 1.0;
Double_t backgroundWeight = 1.0;

// Add signal and background trees
dataloader->AddSignalTree(signalTree, signalWeight);
dataloader->AddBackgroundTree(background, backgroundWeight);

DataSetInfo              : [dataset] : Added class "Signal"
                         : Add Tree RHTree of type Signal with 249278 events
DataSetInfo              : [dataset] : Added class "Background"
                         : Add Tree RHTree of type Background with 249355 events


In [6]:
// Prepare training and testing data
TCut mycuts = "";
TCut mycutb = "";

dataloader->PrepareTrainingAndTestTree(mycuts, mycutb, 
                            "nTrain_Signal=10000:nTrain_Background=10000:SplitMode=Random:NormMode=NumEvents:!V");

### Specify the strings for the CNN

In [7]:
// Input Layout
TString inputLayoutString("InputLayout=1|32|32");

// General layout.
TString layoutString("Layout=CONV|12|2|2|1|1|1|1|TANH,MAXPOOL|6|6|1|1,RESHAPE|1|1|9408|FLAT,DENSE|512|TANH,DENSE|32|"
                     "TANH,DENSE|2|LINEAR");

// Training strategies.
TString training0("LearningRate=1e-1,Momentum=0.9,Repetitions=1,"
                  "ConvergenceSteps=20,BatchSize=256,TestRepetitions=10,"
                  "WeightDecay=1e-4,Regularization=L2,"
                  "DropConfig=0.0+0.5+0.5+0.5, Multithreading=True");
TString training1("LearningRate=1e-2,Momentum=0.9,Repetitions=1,"
                  "ConvergenceSteps=20,BatchSize=256,TestRepetitions=10,"
                  "WeightDecay=1e-4,Regularization=L2,"
                  "DropConfig=0.0+0.0+0.0+0.0, Multithreading=True");
TString training2("LearningRate=1e-3,Momentum=0.0,Repetitions=1,"
                  "ConvergenceSteps=20,BatchSize=256,TestRepetitions=10,"
                  "WeightDecay=1e-4,Regularization=L2,"
                  "DropConfig=0.0+0.0+0.0+0.0, Multithreading=True");
TString trainingStrategyString("TrainingStrategy=");
trainingStrategyString += training0 + "|" + training1 + "|" + training2;

In [8]:
// General Options.
TString cnnOptions("!H:V:ErrorStrategy=CROSSENTROPY:"
                   "WeightInitialization=XAVIERUNIFORM");

// Concatenate all option strings
cnnOptions.Append(":");
cnnOptions.Append(inputLayoutString);

cnnOptions.Append(":");
cnnOptions.Append(layoutString);

cnnOptions.Append(":");
cnnOptions.Append(trainingStrategyString);

cnnOptions.Append(":Architecture=CPU");

### Train the CNN

In [9]:
TString methodTitle = "DL_CPU";
factory->BookMethod(dataloader, TMVA::Types::kDL, methodTitle, cnnOptions);

// Train MVAs using the set of training events
factory->TrainAllMethods();

// Save the output
outputFile->Close();

std::cout << "==> Wrote root file: " << outputFile->GetName() << std::endl;
std::cout << "==> TMVAClassification is done!" << std::endl;

delete factory;
delete dataloader;

Factory                  : Booking method: DL_CPU
                         : 
                         : Parsing option string: 
                         : ... "!H:V:ErrorStrategy=CROSSENTROPY:WeightInitialization=XAVIERUNIFORM:InputLayout=1|32|32:Layout=CONV|12|2|2|1|1|1|1|TANH,MAXPOOL|6|6|1|1,RESHAPE|1|1|9408|FLAT,DENSE|512|TANH,DENSE|32|TANH,DENSE|2|LINEAR:TrainingStrategy=LearningRate=1e-1,Momentum=0.9,Repetitions=1,ConvergenceSteps=20,BatchSize=256,TestRepetitions=10,WeightDecay=1e-4,Regularization=L2,DropConfig=0.0+0.5+0.5+0.5, Multithreading=True|LearningRate=1e-2,Momentum=0.9,Repetitions=1,ConvergenceSteps=20,BatchSize=256,TestRepetitions=10,WeightDecay=1e-4,Regularization=L2,DropConfig=0.0+0.0+0.0+0.0, Multithreading=True|LearningRate=1e-3,Momentum=0.0,Repetitions=1,ConvergenceSteps=20,BatchSize=256,TestRepetitions=10,WeightDecay=1e-4,Regularization=L2,DropConfig=0.0+0.0+0.0+0.0, Multithreading=True:Architecture=CPU"
                         : The following options are set:
  

Warning in <TH1::TH1>: nbins is <=0 - set to nbins = 1
Warning in <TH2::TH2>: nbinsy is <=0 - set to nbinsy = 1
Warning in <TH2F::LabelsOption>: Cannot sort. No labels


Factory                  : [dataset] : Create Transformation "I" with events from all classes.
                         : 
                         : Transformation, Variable selection : 
TFHandler_Factory        : Variable        Mean        RMS   [        Min        Max ]
                         : -----------------------------------------------------------
                         : -----------------------------------------------------------
                         : Ranking input variables (method unspecific)...
IdTransformation         : Ranking result (top variable is best ranked)
                         : -------------------------
                         : Rank : Variable  : Separation
                         : -------------------------
                         : -------------------------
Factory                  : Train method: DL_CPU for Classification
                         : 
                         : Start of deep neural network training on CPU.
                     

Warning in <TH1::TH1>: nbins is <=0 - set to nbins = 1
Warning in <TH2::TH2>: nbinsy is <=0 - set to nbinsy = 1
Warning in <TH2F::LabelsOption>: Cannot sort. No labels


                         :      Epoch |   Train Err.  Test  Err.     GFLOP/s Conv. Steps
                         : --------------------------------------------------------------
